In [ ]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,LambdaCallback
from keras.layers import Input,Dense, Dropout, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.models import Sequential,Model
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tqdm import tqdm
import seaborn as sns
import numpy as np
import itertools 
import cv2
import os
import io

In [ ]:
labels = ['Control', 'Bipolar']

x_train = [] # training images.
y_train  = [] # training labels.
x_test = [] # testing images.
y_test = [] # testing labels.

image_size = 200

for label in labels:
    trainPath = os.path.join('/content/drive/MyDrive/CNNimageClassification/Training',label)
    for file in tqdm(os.listdir(trainPath)):
        image = cv2.imread(os.path.join(trainPath, file),0) # load images in gray.
        image = cv2.bilateralFilter(image, 2, 50, 50) # remove images noise.
        image = cv2.applyColorMap(image, cv2.COLORMAP_BONE) # produce a pseudocolored image.
        image = cv2.resize(image, (image_size, image_size)) # resize images into 150*150.
        x_train.append(image)
        y_train.append(labels.index(label))
    
    testPath = os.path.join('/content/drive/MyDrive/CNNimageClassification/Testing',label)
    for file in tqdm(os.listdir(testPath)):
        image = cv2.imread(os.path.join(testPath, file),0)
        image = cv2.bilateralFilter(image, 2, 50, 50)
        image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
        image = cv2.resize(image, (image_size, image_size))
        x_test.append(image)
        y_test.append(labels.index(label))

x_train = np.array(x_train) / 255.0 # normalize Images into range 0 to 1.
x_test = np.array(x_test) / 255.0

print(x_train.shape)
print(x_test.shape)

In [ ]:
x_train, y_train = shuffle(x_train,y_train, random_state=42) 

y_train = tf.keras.utils.to_categorical(y_train) #One Hot Encoding on the labels
y_test = tf.keras.utils.to_categorical(y_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42) #Dividing the dataset into Training and Validation sets.

print(x_val.shape)

(532, 200, 200, 3)


In [ ]:
# ImageDataGenerator transforms each image in the batch by a series of random translations, rotations, etc.
datagen = ImageDataGenerator(
    rotation_range=10,                        
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True)

# After you have created and configured your ImageDataGenerator, you must fit it on your data.
datagen.fit(x_train)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
Dropout(0.25)
model.add(layers.Dense(32, activation='softmax'))
model.add(layers.Dense(2))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 198, 198, 128)     3584      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 99, 99, 128)      0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 97, 97, 64)        73792     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 46, 46, 64)        36928     
                                                                 
 flatten_3 (Flatten)         (None, 135424)           

In [ ]:
from keras.optimizers import Adam
callbacks = EarlyStopping(monitor='loss', patience=5)
model.compile(optimizer=Adam(lr=0.0001),loss=keras.losses.CategoricalCrossentropy(),metrics=[keras.metrics.CategoricalAccuracy()])
history = model.fit(datagen.flow(x_train, y_train, batch_size=128),validation_data = (x_val,y_val),epochs = 20,callbacks = callbacks)

Epoch 1/20
17/17 [==============================] - 17s 965ms/step - loss: 2.2682 - categorical_accuracy: 0.4836 - val_loss: 3.4073 - val_categorical_accuracy: 0.2143
Epoch 2/20
17/17 [==============================] - 16s 978ms/step - loss: 2.2396 - categorical_accuracy: 0.4972 - val_loss: 2.6488 - val_categorical_accuracy: 0.3684
Epoch 3/20
17/17 [==============================] - 16s 937ms/step - loss: 2.0145 - categorical_accuracy: 0.5694 - val_loss: 2.5071 - val_categorical_accuracy: 0.3743
Epoch 4/20
17/17 [==============================] - 17s 969ms/step - loss: 1.8885 - categorical_accuracy: 0.5603 - val_loss: 2.4073 - val_categorical_accuracy: 0.3853
Epoch 5/20
17/17 [==============================] - 17s 964ms/step - loss: 1.6682 - categorical_accuracy: 0.6762 - val_loss: 1.9791 - val_categorical_accuracy: 0.3947
Epoch 6/20
17/17 [==============================] - 17s 966ms/step - loss: 1.6122 - categorical_accuracy: 0.6996 - val_loss: 1.9666 - val_categorical_accuracy: 0.413